In [3]:
# Import required packages
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [6]:
# Read one of the files for test purposes
df_file_1 = pd.read_csv('combined_data_1.txt', header=None, names=['customer_id', 'rating', 'date_given'])

In [8]:
df_file_1.head()

,customer_id,rating,date_given
0,1:,NaN,NaN
1,1488844,3.0,2005-09-06
2,822109,5.0,2005-05-13
3,885013,4.0,2005-10-19
4,30878,4.0,2005-12-26


In [9]:
# Drop the date_given column for now and convert rating column to float
df_file_1.drop(columns=['date_given'])

df_file_1['rating'] = df_file_1['rating'].astype('float')

In [42]:
%%time


# Find the indices of NaN values in the 'rating' column
nan_indices = df_file_1[df_file_1['rating'].isnull()].index

# Initialize movie_np array with zeros
movie_np = np.zeros(len(df_file_1), dtype=int)

# Assign movie ids to consecutive NaN values
movie_id = 1
prev_index = nan_indices[0]
for index in nan_indices[1:]:
    movie_np[prev_index+1:index] = movie_id
    movie_id += 1
    prev_index = index

# Assign movie id to the remaining NaN values after the last NaN index
movie_np[nan_indices[-1]+1:] = movie_id

# Remove leading zeros if any
movie_np = movie_np[movie_np != 0]

CPU times: user 114 ms, sys: 150 ms, total: 264 ms
Wall time: 261 ms


In [33]:
df_nan

,rating
0,True
1,False
2,False
3,False
4,False
...,...
24058258,False
24058259,False
24058260,False
24058261,False


In [34]:
df_file_1[df_nan['rating']==True]

,customer_id,rating,date_given
0,1:,NaN,NaN
548,2:,NaN,NaN
694,3:,NaN,NaN
2707,4:,NaN,NaN
2850,5:,NaN,NaN
...,...,...,...
24046714,4495:,NaN,NaN
24047329,4496:,NaN,NaN
24056849,4497:,NaN,NaN
24057564,4498:,NaN,NaN


In [ ]:



    # numpy approach
    

# Account for last record and corresponding length
# numpy approach


print('Movie numpy: {}'.format(movie_np))
print('Length: {}'.format(len(movie_np)))